In [ ]:
# one hot encode the target variable

In [3]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
#from googletrans import Translator
from translate import Translator
from deep_translator import GoogleTranslator
pd.set_option("display.max_columns", None)

In [31]:
train = pd.read_csv('data/train_translated.csv')

In [4]:
# suffisticated approach for translation
def translate(data, column):
    original = data[column].dropna().unique()
    translated = [GoogleTranslator(source='et', target='en').translate(word) for word in original]
    print('done')
    data[column] = data[column].replace(original, translated)

In [5]:
translate(train, 'state')

done


In [27]:
train = train.drop(['Unnamed: 0'], axis=1)

In [28]:
train.set_index('id', inplace=True)

In [30]:
train.to_csv('data/train_translated.csv')

In [32]:
train

,id,full_nr,name,ks,material,commentary,event_type,location,start,end,before_Christ,country_and_unit,participants_role,participant,parish,text,class,technique,parameter,unit,value,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,collection_queue_nr,collection_additional_nr,element_count,legend,is_original,initial_info,damages,state,color,additional_text,type
0,232170,ETMM _ 12150:115 Aj 118:44/M20,"Kuno Areng, Bremerhaven Festwoche medal",118.0,metal,NaN,festivals,the city of Bremerhaven,1979,NaN,ei,Germany,participant,"Areng, Kuno",NaN,NaN,NaN,NaN,diameter,cm,4,ETMM,_,12150.0,115.0,NaN,Aj,44.0,M20,1.0,NaN,1.0,Festwoche - Breemenhaven,NaN,good,grey,KUTTER ASTARTE -SCHIFFERGILDE BREMENHAVEN E.V.,medal
1,2251378,ETMM _ 12584:19 M 102:1/13:13,"Photo-Villem Kapp, photo with dedication to Ar...",102.0,photographic material,NaN,filmmaking and photography,NaN,1938,NaN,ei,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETMM,_,12584.0,19.0,NaN,M,1.0,13:13,1.0,Photos from the collection of Villem Kapi and ...,NaN,NaN,NaN,good,NaN,NaN,photo
2,2070466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,F,52.0,NaN,1.0,NaN,NaN,NaN,NaN,good,NaN,NaN,photo
3,4085096,ETMM _ Fk 41691/k,"Metspart, Youth Theater, 1969, in parts: Hedvi...",41691.0,photographic material,NaN,theater,NaN,18.09.1969,NaN,ei,NaN,liaison organization,Noorsooteater,NaN,NaN,NaN,NaN,film frame,60 x 60 mm,1,ETMM,_,NaN,NaN,NaN,Fk,NaN,k,1.0,NaN,1.0,TB080321,NaN,satisfactory,NaN,NaN,photo negative
4,2697904,ETMM _ 9424 Mo 238:1/62:05,Letter: Folk musicians: Violin: Jüri Saal: Let...,238.0,paper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,manuscript,NaN,NaN,NaN,ETMM,_,9424.0,NaN,NaN,Mo,1.0,62:05:00,1.0,Folk instrumentalists Mo238,1.0,NaN,NaN,satisfactory,NaN,NaN,a letter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,2449318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETMM,_,8291.0,NaN,NaN,T,1.0,56:65,1.0,NaN,1.0,NaN,NaN,good,NaN,NaN,plan
13996,2974108,ETMM _ 5429 T 10:3/244:1,"Photo, Maskiball, Theater Estonia",10.0,photo material>photo paper,NaN,taking pictures,NaN,NaN,NaN,NaN,NaN,the author,J. & P. Parikas,NaN,NaN,NaN,photography>black and white photography,height,cm,"8,7",ETMM,_,5429.0,NaN,NaN,T,3.0,244:01:00,1.0,NaN,1.0,NaN,glue marks on the back,good,NaN,NaN,photo
13997,2068347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,F,12.0,NaN,1.0,NaN,NaN,NaN,NaN,good,NaN,NaN,photo
13998,3481436,AM _ 8986:1 G 1678:1,graphics by Ursula Elisabeth Borcke. A portrai...,1678.0,paper,NaN,portrait of a woman,NaN,1730,NaN,ei,NaN,engraver,"Wolfgang, Georg Andreas vanem",NaN,NaN,NaN,copper engraving,height,cm,53,AM,_,8986.0,1.0,NaN,G,1.0,NaN,1.0,NaN,1.0,NaN,"heavily soiled, creased, with creases, tears",satisfactory,NaN,NaN,graphics
